In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model_name = "microsoft/phi-2"
phi2_model_pretrained = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Create model 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Phi2wrapper(nn.Module):
    
    #This defines the structure of the NN.
    def __init__(self, input_dim_CLIP=768, input_dim_phi2=2560, phi2_model=phi2_model_pretrained):
        super(Phi2wrapper, self).__init__()
        self.input_dim_CLIP = 768
        self.input_dim_phi2 = 2560
        self.projection_img = nn.Linear(self.input_dim_CLIP, self.input_dim_phi2, 
                                        bias=False)
        self.phi2_model = phi2_model

    def forward(self, x):
        x = self.projection_img(x)
        x = self.phi2_model(x)
        return x 

In [9]:
phi2_projection_model = Phi2wrapper()

In [12]:
## Freezing phi-2 for projection layer training 

for name, param in phi2_projection_model.named_parameters():
    if "phi2_model" in name:
        param.requires_grad = False

### Create dataset

In [59]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd 
import json
import os 
import h5py


In [60]:
def get_image_name(image_id_from_caption, list_image_info): 
    for img in list_image_info: 
        if img['id'] == image_id_from_caption: 
            img_name = img['file_name'].split('.')[0]
            return img['file_name'].split('.')[0]
    return 'NoImgNameFound'

In [65]:
# file_path_captions_coco = '/media/App/amaranth/lavanya/Capstone_data/annotations_trainval2017/annotations/captions_train2017.json'

# with open(file_path_captions_coco) as f:
#    data = json.load(f)

# captions_info = []
# for a in data['annotations']: 
#     captions_info.append([a['image_id'], a['caption'], a['id']])

# captions_info_df = pd.DataFrame(data=captions_info, columns=['image_id', 'caption', 'caption_id'])
# captions_info_df['image_name'] = captions_info_df['image_id'].apply(lambda x: get_image_name(x, data['images']))
# captions_info_df['image_name'] = captions_info_df['image_name'].apply(lambda x: '0'*(12-len(str(x))) + str(x))
# captions_info_df.to_csv('captions_images_map_COCO_train2017.csv')

In [76]:
captions_info_df = pd.read_csv('captions_images_map_COCO_train2017.csv')

/tmp/ipykernel_3065095/1237926302.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  captions_info_df = pd.read_csv('captions_images_map_COCO_train2017.csv')


In [83]:
import h5py    
import numpy as np    

In [ ]:
class COCO_CLIP_Dataset(Dataset):
    def __init__(
        self, caption_file, embedding_path):
        self.embedding_path = embedding_path
        self.caption_file = caption_file

    def __len__(self):
        return len(self.caption_file)
    
    def __getitem__(self, index):
        row = self.caption_file.iloc[[index]]

        img_base_name = '0'*(12-len(str(row['image_id']))) + str(row['image_id'])
        img_clip_embedding_path = os.path.join(self.embedding_path, f'{img_base_name}.h5')  
        np_array_embed_img = h5py.File(img_clip_embedding_path,'r+')['image_features'][()]
        
        img_caption = row['caption'] ## Tokenize this 

        return torch.tensor(np_array_embed_img), img_caption